# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**

**Fecha**: 04/04/4045

**Nombre del equipo**: Equipo 2.0

**Profesor**: Pablo Camarillo Ramirez

In [88]:
import findspark
findspark.init()

In [89]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://34ad4bc7c499:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Java Listener

In [90]:
from equipo.log_listener import TrafficListener
spark.streams.addListener(TrafficListener())

In [91]:
log_lines = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/log_streaming/input/")


25/04/04 17:13:00 ERROR MicroBatchExecution: Query [id = e4e5b7df-c42e-4f3e-917b-d16df8d1bdde, runId = fd6aa758-1315-4f86-9839-8399038a6691] terminated with error
java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.ja

In [92]:
from pyspark.sql.functions import split

log_df = log_lines.select(split(log_lines.value, " | ").alias("logs_array"))
log_df = log_df.withColumn("timestamp", log_df["logs_array"].getItem(0).cast("string"))
log_df = log_df.withColumn("level", log_df["logs_array"].getItem(1).cast("string"))
log_df = log_df.withColumn("message", log_df["logs_array"].getItem(2).cast("string"))
log_df = log_df.withColumn("server", log_df["logs_array"].getItem(3).cast("string"))

log_df.printSchema()


root
 |-- logs_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- timestamp: string (nullable = true)
 |-- level: string (nullable = true)
 |-- message: string (nullable = true)
 |-- server: string (nullable = true)



In [93]:
log_df = log_df.filter(log_df['level'] == "ERROR")
log_df.printSchema()

root
 |-- logs_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- timestamp: string (nullable = true)
 |-- level: string (nullable = true)
 |-- message: string (nullable = true)
 |-- server: string (nullable = true)



In [ ]:
query_files = log_df \
                .writeStream \
                .outputMode("append") \
                .format("console") \
                .trigger(processingTime='10 seconds') \
                .option("path", "/home/jovyan/notebooks/data/log_streaming/output/") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .start()
query_files.awaitTermination(60)

Query started: e4e5b7df-c42e-4f3e-917b-d16df8d1bdde


25/04/04 17:13:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 83
-------------------------------------------
+----------+---------+-----+-------+------+
|logs_array|timestamp|level|message|server|
+----------+---------+-----+-------+------+
+----------+---------+-----+-------+------+

Query made progress: {
  "id" : "e4e5b7df-c42e-4f3e-917b-d16df8d1bdde",
  "runId" : "8c4d6383-1cde-410f-8af8-6b8933561437",
  "name" : null,
  "timestamp" : "2025-04-04T17:13:05.327Z",
  "batchId" : 83,
  "numInputRows" : 37,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 14.925373134328357,
  "durationMs" : {
    "addBatch" : 2313,
    "commitOffsets" : 55,
    "getBatch" : 61,
    "queryPlanning" : 17,
    "triggerExecution" : 2479
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/home/jovyan/notebooks/data/log_streaming/input]",
    "startOffset" : {
      "logOffset" : 82
    },
    "endOffset" : {
      "logOffset" : 83
    },
    "latestOffset" : null,
   

25/04/04 17:13:15 WARN FileStreamSource: Listed 291 file(s) in 5042 ms


-------------------------------------------
Batch: 84
-------------------------------------------
+----------+---------+-----+-------+------+
|logs_array|timestamp|level|message|server|
+----------+---------+-----+-------+------+
+----------+---------+-----+-------+------+

Query made progress: {
  "id" : "e4e5b7df-c42e-4f3e-917b-d16df8d1bdde",
  "runId" : "8c4d6383-1cde-410f-8af8-6b8933561437",
  "name" : null,
  "timestamp" : "2025-04-04T17:13:10.000Z",
  "batchId" : 84,
  "numInputRows" : 35,
  "inputRowsPerSecond" : 7.4898352236250805,
  "processedRowsPerSecond" : 6.271277548826375,
  "durationMs" : {
    "addBatch" : 283,
    "commitOffsets" : 56,
    "getBatch" : 73,
    "latestOffset" : 5101,
    "queryPlanning" : 11,
    "triggerExecution" : 5581,
    "walCommit" : 56
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/home/jovyan/notebooks/data/log_streaming/input]",
    "startOffset" : {
      "logOffset" : 83
    },
    "endOffset" : 

False

In [95]:
speed_df = spark \
            .read \
            .parquet("/home/jovyan/notebooks/data/log_streaming/output/")

speed_df.show(4, False)

+----------+---------+-----+-------+------+
|logs_array|timestamp|level|message|server|
+----------+---------+-----+-------+------+
+----------+---------+-----+-------+------+



In [96]:
sc.stop()

25/04/04 17:14:11 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 90, writer: ConsoleWriter[numRows=20, truncate=true]] is aborting.
25/04/04 17:14:11 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 90, writer: ConsoleWriter[numRows=20, truncate=true]] aborted.
25/04/04 17:14:11 ERROR MicroBatchExecution: Query [id = e4e5b7df-c42e-4f3e-917b-d16df8d1bdde, runId = 8c4d6383-1cde-410f-8af8-6b8933561437] terminated with error
org.apache.spark.SparkException: Job 10 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet$Node.foreach(HashSet.scala:435)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:361)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAG